In [1]:
import numpy as np
import math, json, time, types, copy, sys, os
import torch
from torch.nn import functional as F
import torch.nn as nn

from transformers import PreTrainedTokenizerFast

np.set_printoptions(precision=4, suppress=True, linewidth=200)

In [2]:
RUN_DEVICE = 'cpu'
ctx_len = 768
n_layer = 24
n_embd = 1024

MODEL_NAME = '/data1/ckw/20220615-10803' #修改为自己的模型路径

vocab_size = 50277
VOCAB_NAME = '20B_tokenizer.json'

print(f'\n* running on {RUN_DEVICE}')


* running on cpu


### 什么是RWKV？

RWKV是Receptance Weighted Key Value的缩写，是一种结合了RNN（循环神经网络）和Transformer优势的新型神经网络架构。它的设计目的是解决Transformer在处理长序列时的内存和计算复杂度问题，同时保留RNN在推理阶段的计算效率。RWKV利用线性注意力机制，可以将其形式化为Transformer或RNN，从而在训练期间实现并行计算，并在推理过程中保持恒定的计算和内存复杂度。

RWKV 的 ChannelMix 实现方式结合了时间混合和通道混合的操作。下面是对代码和其对应公式的详细解释：

1. **时间混合（Time Mixing）**：
   时间混合通过 `time_mix` 参数和 `time_shift` 操作来实现。这一步的目的是结合当前时间步的输入和前一个时间步的输入。

   公式表示：

   
   \begin{align*}
   x' = x \cdot \text{time\_mix} + \text{time\_shift}(x) \cdot (1 - \text{time\_mix})
   \end{align*}

   其中，`time_shift` 操作是一个时间步的移位操作，`time_mix` 是一个可训练的参数。

3. **键（Key）生成**：
   使用一个线性层 `self.key` 将输入 `x'` 转换成键 `k`，然后应用 ReLU 激活函数和平方操作。

   公式表示：
   \begin{align*}
   k = \text{ReLU}(\text{key}(x'))^2
   \end{align*}

4. **值（Value）生成**：
   将键 `k` 输入到值线性层 `self.value` 以生成值 `kv`。

   公式表示：
   \begin{align*}
   kv = \text{value}(k)
   \end{align*}

5. **接收函数（Receptance Function）**：
   使用一个线性层 `self.receptance` 计算接收函数 `r`，然后应用 Sigmoid 激活函数。

   公式表示：
   \begin{align*}
   r = \sigma(\text{receptance}(x'))
   \end{align*}

6. **最终输出**：
   将接收函数 `r` 与值 `kv` 相乘生成最终输出 `rkv`。

   公式表示：
   \begin{align*}
   rkv = r \cdot kv
   \end{align*}

综合这些步骤，整个 ChannelMix 的计算过程可以用以下公式表示：

\begin{align*}
x' & = x \cdot \text{time\_mix} + \text{time\_shift}(x) \cdot (1 - \text{time\_mix}) \\
k & = \text{ReLU}(\text{key}(x'))^2 \\
kv & = \text{value}(k) \\
r & = \sigma(\text{receptance}(x')) \\
\text{output} & = r \cdot kv
\end{align*}


以上公式解释了 RWKV 的 ChannelMix 实现的细节。

In [3]:
class RWKV_ChannelMix(nn.Module):
    def __init__(self, layer_id):
        super().__init__()
        self.layer_id = layer_id

        self.time_shift = nn.ZeroPad2d((0,0,1,-1))
        self.time_mix = nn.Parameter(torch.ones(1, 1, n_embd))

        hidden_sz = 4 * n_embd
        self.key = nn.Linear(n_embd, hidden_sz, bias=False)
        self.receptance = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(hidden_sz, n_embd, bias=False)

    def forward(self, x):
        x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)

        k = self.key(x)
        k = torch.square(torch.relu(k))
        kv = self.value(k)
        
        rkv = torch.sigmoid(self.receptance(x)) * kv
        return rkv

在RWKV的实现中，`RWKV_TimeMix` 通过时间混合来处理输入数据。下面是具体的实现和对应的公式说明：

1. **时间混合（Time Mixing）**：
   时间混合是通过 `time_mix` 参数和 `time_shift` 操作来实现的。这一步的目的是结合当前时间步的输入和前一个时间步的输入。

   公式表示：
   \begin{align*}
   x' = x \cdot \text{time\_mix} + \text{time\_shift}(x) \cdot (1 - \text{time\_mix})
   \end{align*}

2. **键（Key）生成**：
   使用一个线性层 `self.key` 将输入 `x'` 转换成键 `k`，然后对其进行转置。

   公式表示：
   \begin{align*}
   k = \text{key}(x')^T
   \end{align*}

3. **值（Value）生成**：
   使用一个线性层 `self.value` 将输入 `x'` 转换成值 `v`，然后对其进行转置。

   公式表示：
   \begin{align*}
   v = \text{value}(x')^T
   \end{align*}

4. **接收函数（Receptance Function）**：
   使用一个线性层 `self.receptance` 计算接收函数 `r`。

   公式表示：
   \begin{align*}
   r = \text{receptance}(x')
   \end{align*}

5. **键值相乘**：
   将键 `k` 和值 `v` 相乘得到 `kv`。

   公式表示：
   \begin{align*}
   kv = k \cdot v
   \end{align*}

6. **时间权重计算**：
   计算时间权重 `w`，其中 `self.time_w` 是通过 `time_decay` 和 `time_curve` 计算得到的。

   公式表示：
   \begin{align*}
   \text{self.time\_w} &= \exp(\text{time\_decay}) \cdot \text{time\_curve} \\
   w &= \exp(\text{self.time\_w})
   \end{align*}

7. **卷积操作**：
   使用一维卷积计算加权键值和加权键。

   公式表示：
   \begin{align*}
   wkv &= \text{conv1d}(\text{ZeroPad2d}(kv), w, \text{groups}=C) \\
   wk &= \text{conv1d}(\text{ZeroPad2d}(k), w, \text{groups}=C) + 1e-9
   \end{align*}

8. **最终输出**：
   将接收函数 `r` 与加权键值比 `wkv / wk` 相乘，并通过输出线性层得到最终输出 `rwkv`。

   公式表示：
   \begin{align*}
   rwkv &= \sigma(r) \cdot \left( \frac{wkv}{wk} \right)^T \\
   rwkv &= \text{output}(rwkv)
   \end{align*}

综合这些步骤，`RWKV_TimeMix` 的整个计算过程可以表示为：

\begin{align*}
x' &= x \cdot \text{time\_mix} + \text{time\_shift}(x) \cdot (1 - \text{time\_mix}) \\
k &= \text{key}(x')^T \\
v &= \text{value}(x')^T \\
r &= \text{receptance}(x') \\
kv &= k \cdot v \\
\text{self.time\_w} &= \exp(\text{time\_decay}) \cdot \text{time\_curve} \\
w &= \exp(\text{self.time\_w}) \\
wkv &= \text{conv1d}(\text{ZeroPad2d}(kv), w, \text{groups}=C) \\
wk &= \text{conv1d}(\text{ZeroPad2d}(k), w, \text{groups}=C) + 1e-9 \\
rwkv &= \sigma(r) \cdot \left( \frac{wkv}{wk} \right)^T \\
rwkv &= \text{output}(rwkv)
\end{align*}


In [ ]:
class RWKV_TimeMix(nn.Module):
    def __init__(self, layer_id):
        super().__init__()
        self.layer_id = layer_id
        self.time_decay = nn.Parameter(torch.ones(n_embd, 1))
        self.time_curve = torch.tensor([-(ctx_len - 2 - i) for i in range(ctx_len-1)]).unsqueeze(0)
        self.time_first = nn.Parameter(torch.ones(n_embd, 1) * math.log(0.3))
        
        self.time_shift = nn.ZeroPad2d((0,0,1,-1))
        self.time_mix = nn.Parameter(torch.ones(1,1,n_embd))

        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)
        self.receptance = nn.Linear(n_embd, n_embd, bias=False)

        self.output = nn.Linear(n_embd, n_embd, bias=False)

    def forward(self, x):
        B, T, C = x.size()

        x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)

        k = self.key(x).transpose(-1, -2)
        v = self.value(x).transpose(-1, -2)
        r = self.receptance(x)

        k = torch.clamp(k, max=60)
        k = torch.exp(k)

        kv = k * v

        self.time_w = torch.cat([torch.exp(self.time_decay) * self.time_curve.to(self.time_decay.device), self.time_first], dim=-1)
        w = torch.exp(self.time_w)
        
        w = w[:,-T:].unsqueeze(1)
        wkv = F.conv1d(nn.ZeroPad2d((T-1, 0, 0, 0))(kv), w, groups=C)
        wk = F.conv1d(nn.ZeroPad2d((T-1, 0, 0, 0))(k), w, groups=C) + 1e-9

        rwkv = torch.sigmoid(r) * (wkv / wk).transpose(-1, -2)
        
        rwkv = self.output(rwkv)
        return rwkv

### RWKV的Block

RWKV的Block是一个基本的模块，它结合了时间混合（TimeMix）和通道混合（ChannelMix）操作。Block中的每个模块（时间混合和通道混合）都通过归一化和残差连接来处理输入数据，从而增强模型的稳定性和性能。

### 主要组件和操作

1. **LayerNorm**：用于归一化输入，增强训练的稳定性。
   - `self.ln1` 和 `self.ln2` 分别在时间混合和通道混合之前对输入进行归一化。
   
2. **时间混合（TimeMix）**：结合当前时间步和前一个时间步的信息，捕获时间依赖性。
   - `self.att = RWKV_TimeMix(layer_id)` 初始化时间混合模块。
   
3. **通道混合（ChannelMix）**：在不同通道间进行混合，增强模型的表达能力。
   - `self.ffn = RWKV_ChannelMix(layer_id)` 初始化通道混合模块。
   
4. **残差连接**：通过将混合操作的输出加回到原始输入上，保持信息流动并增强模型的梯度传播能力。

通过这种设计，RWKV的Block能够高效地处理序列数据，结合时间和通道信息，提高模型的表现。

In [4]:
class Block(nn.Module):
    def __init__(self, layer_id):
        super().__init__()
        self.layer_id = layer_id  # 存储当前层的ID

        # 定义两个LayerNorm层，用于归一化输入
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
        # 定义时间混合和通道混合模块
        self.att = RWKV_TimeMix(layer_id)
        self.ffn = RWKV_ChannelMix(layer_id)

    def forward(self, x):
        # 首先对输入进行LayerNorm归一化
        x = self.ln1(x)
        
        # 进行时间混合操作，并通过残差连接将结果加回到输入上
        x = x + self.att(x)
        
        # 再次对输入进行LayerNorm归一化
        x = self.ln2(x)
        
        # 进行通道混合操作，并通过残差连接将结果加回到输入上
        x = x + self.ffn(x)
        
        # 返回最终的输出
        return x


接下来，实现了RWKV模型的主要部分：

1. **模型加载和预处理**：代码中加载模型权重并进行时间相关权重的预处理。
2. **LayerNorm**：在`LN`方法中实现了层归一化，关于LayerNorm的使用。
3. **前馈网络（FF）和自注意力（SA）**：`FF`方法实现了前馈网络的计算，`SA`方法实现了自注意力机制的计算。这两部分对应TimeMix和ChannelMix的详细计算。
4. **运行模型**：`run`方法实现了模型的整体运行逻辑，依次通过每一层，并最终输出结果。即模型的运行和推理过程。

In [5]:
time_buf = {}  # 用于缓存时间相关信息的全局字典

class RWKV_RNN():
    def __init__(self, MODEL_NAME=MODEL_NAME):
        print('\nloading RWKV-RNN', MODEL_NAME)
        self.ctx_len = ctx_len  # 上下文长度
        self.n_layer = n_layer  # 网络层数
        self.n_embd = n_embd    # 嵌入维度
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=VOCAB_NAME)  # 初始化分词器

        self.w = types.SimpleNamespace()  # 用于存储模型权重的命名空间
        
        # 加载模型权重文件
        w = torch.load(MODEL_NAME + '.pth', map_location=torch.device(RUN_DEVICE))

        # 处理时间相关的权重
        for x in w.keys():
            if '.time_' in x:
                w[x] = w[x].squeeze()  # 压缩维度
            if '.time_decay' in x:
                w[x] = torch.exp(-torch.exp(w[x]))  # 对时间衰减进行双重指数运算
            if '.time_first' in x:
                w[x] = torch.exp(w[x])  # 对时间初始值进行指数运算
                    
            # 将权重存储在命名空间中
            xx = x.split('.')
            here = self.w
            for i in range(len(xx)):
                if xx[i].isdigit():
                    ii = int(xx[i])
                    if ii not in here:
                        here[ii] = types.SimpleNamespace()  # 初始化命名空间
                    here = here[ii]
                else:
                    if i == len(xx) - 1:
                        setattr(here, xx[i], w[x])
                    elif not hasattr(here, xx[i]):
                        if xx[i+1].isdigit():
                            setattr(here, xx[i], {})
                        else:
                            setattr(here, xx[i], types.SimpleNamespace())
                    here = getattr(here, xx[i])
    
        self.clear()  # 初始化缓存
    
    def clear(self):
        self.xx = {}  # 清空缓存
        self.aa = {}
        self.bb = {}
    
    def save(self, target):
        # 深拷贝当前状态到目标
        target.xx = copy.deepcopy(self.xx)
        target.aa = copy.deepcopy(self.aa)
        target.bb = copy.deepcopy(self.bb)
    
    def load(self, target):
        # 从目标深拷贝状态到当前实例
        self.xx = copy.deepcopy(target.xx)
        self.aa = copy.deepcopy(target.aa)
        self.bb = copy.deepcopy(target.bb)

    def LN(self, xx, w):
        # 执行LayerNorm归一化
        return F.layer_norm(xx, (n_embd,), weight=w.weight, bias=w.bias)

    def FF(self, xx, w, name):
        # 前馈网络计算
        if name not in self.xx:
            self.xx[name] = torch.zeros(n_embd, device=RUN_DEVICE)
        x = xx * w.time_mix + self.xx[name] * (1 - w.time_mix)  # 混合当前输入和缓存

        self.xx[name] = xx  # 更新缓存

        r = torch.sigmoid(w.receptance.weight @ x)  # 计算接收向量
        k = torch.square(torch.relu(w.key.weight @ x))  # 计算键向量
        kv = w.value.weight @ k  # 计算值向量

        return r * kv  # 返回前馈网络输出

    def SA(self, xx, w, name):
        # 自注意力计算
        if name not in self.xx:
            self.xx[name] = torch.zeros(n_embd, device=RUN_DEVICE)
            self.aa[name] = torch.zeros(n_embd, device=RUN_DEVICE)
            self.bb[name] = torch.zeros(n_embd, device=RUN_DEVICE)
        x = xx * w.time_mix + self.xx[name] * (1 - w.time_mix)  # 混合当前输入和缓存
        self.xx[name] = xx  # 更新缓存

        r = torch.sigmoid(w.receptance.weight @ x)  # 计算接收向量

        k = torch.exp(torch.clamp(w.key.weight @ x, max=60))  # 计算键向量
        v = w.value.weight @ x  # 计算值向量
        kv = k * v  # 计算键值对

        a = self.aa[name] + w.time_first * kv  # 计算新的a值
        b = self.bb[name] + w.time_first * k  # 计算新的b值
        self.aa[name] = w.time_decay * self.aa[name] + kv  # 更新缓存中的a值
        self.bb[name] = w.time_decay * self.bb[name] + k  # 更新缓存中的b值

        rwkv = r * a / (b + 1e-9)  # 计算自注意力输出

        return w.output.weight @ rwkv  # 返回自注意力输出

    def run(self, ctx):
        # 运行模型
        w = self.w
        x = w.emb.weight[ctx[-1]]  # 获取当前token的嵌入

        # 依次通过每一层
        for i in range(n_layer):
            x = self.LN(x, w.blocks[i].ln1)  # 归一化
            x = x + self.SA(x, w.blocks[i].att, f'att.{i}')  # 自注意力计算并残差连接
            x = self.LN(x, w.blocks[i].ln2)  # 归一化
            x = x + self.FF(x, w.blocks[i].ffn, f'ffn.{i}')  # 前馈网络计算并残差连接

        x = self.LN(x, w.ln_out)  # 最后一层归一化

        x = w.head.weight @ x  # 计算输出
        x = x.tolist()  # 转换为列表

        return x  # 返回最终结果

In [6]:
print('''
******************************************************************************
* This is a preview of RWKV-v2-RNN trained on the Pile for only 50B tokens.
* It is NOT indicative of the final performance (which requires 300B tokens).
******************************************************************************''')


******************************************************************************
* This is a preview of RWKV-v2-RNN trained on the Pile for only 50B tokens.
* It is NOT indicative of the final performance (which requires 300B tokens).
******************************************************************************


In [7]:
# Edit model.py to set CPU / CUDA mode. Runs on CPU by default.

TEMPERATURE = 1.0
TOP_P = 0.7

DEBUG_DEBUG = False
LENGTH_OF_EACH = 333
NUM_TRIALS = 3

context = '\nDataWhalechina is an organization founded at Shanghai Jiao Tong University that helps learners learn artificial intelligence.'

##############################################################################################################

In [8]:
model = RWKV_RNN()


loading RWKV-RNN /data1/ckw/20220615-10803


下面我们从给定的输出logits中进行采样，以生成一个新的token。它实现了**温度调节采样**和**核采样（Top-p采样）**，具体步骤如下：

1. **Softmax转换**：将模型输出的logits通过softmax函数转换为概率分布。
2. **排序和累积概率计算**：对概率从高到低进行排序，并计算累积概率分布。
3. **核采样**：
   - 计算累积概率超过`top_p`的最小值，确定截断值`cutoff`。
   - 将所有低于截断值的概率置为0，从而保留最重要的`top_p`部分概率。
4. **温度调节**：如果`temperature`不为1，则调整概率分布，使得概率分布更平滑或更尖锐。
5. **采样**：从调整后的概率分布中采样一个值，返回对应的索引。

这种方法在文本生成任务中尤为常用，通过调节`temperature`和`top_p`参数，可以控制生成文本的多样性和质量。

In [9]:
def sample_logits(out, temperature=1.0, top_p=None):
    # 将输出转化为概率分布（通过softmax函数）
    probs = F.softmax(torch.tensor(out), dim=-1)
    
    # 按概率从高到低排序
    sorted_probs, _ = torch.sort(probs, descending=True)

    # 计算累积概率分布
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1).numpy()
    
    # 根据累积概率和top_p计算截断值（cutoff）
    cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])
    
    # 将低于截断值的概率置为0
    probs[probs < cutoff] = 0

    # 如果temperature不等于1，则对概率进行温度调节
    if temperature != 1.0:
        probs = probs.pow(1.0 / temperature)

    # 从调整后的概率分布中采样一个值并返回
    return torch.multinomial(probs, num_samples=1)[0]


In [10]:
for TRIAL in range(1 if DEBUG_DEBUG else NUM_TRIALS):
    ctx = [model.tokenizer.encode(context)][0]
    src_len = len(ctx)
    print(context, end='')

    model.clear()
    if TRIAL == 0:
        init_state = types.SimpleNamespace()
        for i in range(src_len if DEBUG_DEBUG else src_len):
            x = ctx[:i+1]
            if i == src_len - 1:
                init_state.out = model.run(x)
            else:
                model.run(x)
        model.save(init_state)
    else:
        model.load(init_state)

    if DEBUG_DEBUG:
        out = init_state.out
        print('\n', np.array(x), '==>', np.array(
            out), np.max(out), np.min(out))

    for i in range(src_len, src_len + (0 if DEBUG_DEBUG else LENGTH_OF_EACH)):
        x = ctx[:i+1]
        x = x[-model.ctx_len:]

        if i == src_len:
            out = copy.deepcopy(init_state.out)
        else:
            out = model.run(x)

        out[0] = -999999999  # disable <|endoftext|>

        char = sample_logits(out, temperature=TEMPERATURE, top_p=TOP_P)
        char = char.item()
        print(model.tokenizer.decode(char), end='', flush=True)

        ctx += [char]
    print('\n' + '-' * 70, end='')


DataWhalechina is an organization founded at Shanghai Jiao Tong University that helps learners learn artificial intelligence. We want to change the way students learn artificial intelligence. We are very committed to the idea of artificial intelligence. We have already taken part in a joint research project with the European Research Council. This will bring us closer to our goal. We hope that this research project will give us the opportunity to develop a framework for a data-driven approach in artificial intelligence.

I.C. Pfeifer

Research Fellow

Our work in data science and machine learning has a strong connection with the Human Brain Project, an initiative of the University of California at Berkeley. The focus of our work is on the research of language and language learning, with an emphasis on language acquisition. Our research is directed at the development of technology to improve the language acquisition skills of students and their parents.

The Joint Data-Science-Technolo